# Classification

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedGroupKFold

from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, matthews_corrcoef
from scipy.stats import mode
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer,  confusion_matrix, roc_auc_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from scikeras.wrappers import KerasClassifier
from keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

import joblib 

import random
np.random.seed(123)
random.seed(123)
tf.random.set_seed(123)

Upload scaled data file and extrated data file obtained using components from the PCA, MFA and t-SNE methods. Both data files are then merged into a single dataframe named all_data.

In [3]:
#Upload CSV files
scaled_data = pd.read_csv("Data/scaled_data.csv")
extracted_data = pd.read_csv("Data/df_extracted_features.csv")

# Concatenate all data in a single dataframe:
all_data = pd.concat([scaled_data, extracted_data], axis=1)

Load the various lists of selected variables generated by the different methods and components created through the various feature extraction techniques:

In [4]:
# Import generated and selected variables: 
import sys
sys.path.append("Data/") 
from feature_extraction import pca_features, mfa_features, tsne_features
from feature_selection import mrmr_features, rfe_features, ga_features

# Define sets of variables to be evaluated
all_features = [mrmr_features, rfe_features, ga_features, pca_features, mfa_features, tsne_features]
all_names = ["mRMR_Features", "RFE_Features","GA_Features" ,"PCA_Features", "MFA_Features", "tSNE_Features"]
Class = scaled_data.columns[754]

Each subset of variables/components obtained in the variable reduction step will be used in different classification models. To evaluate different techniques for the classification of the subjects into the two groups, two functions are defined:    
A `classification` function trains and evaluates different sets of variables using a specified classifier. The evaluation process employs the Leave-One-Group-Out cross-validation method. If a parameter grid is provided, it performs an hyperparameter tuning to maximize the Matthews Correlation Coefficient (MCC) score in the validation data.    
A range of performance metrics are calculated with the `metrics_calculation` function. Given that each subject is represented by three different observations, a voting process is applied to classify the subject into either the Parkinson's or healthy group. This process aggregates the predictions from all three observations to determine the final classification for each subject. The majority vote from the individual observations will dictate the subject's group classification. Then different metrics are calculated, including Accuracy, F1-score, Area Under the Curve (AUC), MCC, Kappa, Sensitivity, Specificity, Positive Predictive Value (PPV) and Negative Predictive Value (NPV).    
Finally, the `classification` function returns two dataframes, one with the metrics obtained during the training process, and the second one with the metrics obtained for the test set (validation data).

In [5]:
def metrics_calculation(labels, predictions, probabilities, grid_search=None):
    '''
    Calculates evaluation metrics: Accuracy, F1-score, AUC, MCC, Kappa, Sensitivity, Specificity, PPV and NPV.
    Optionally includes best parameters if GridSearch is used.
    '''
    # Voting final predictions
    final_pred = []
    for i in range(0, len(labels), 3):  # 3 observations per subject
        patient_preds = predictions[i:i + 3]
        final = mode(patient_preds).mode.item()
        final_pred.append(final)

    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels[::3], final_pred).ravel()
        
    # Metrics calculation
    accuracy = accuracy_score(labels[::3], final_pred)
    f1 = f1_score(labels[::3], final_pred, average='weighted')
    kappa = cohen_kappa_score(labels[::3], final_pred)
    mcc = matthews_corrcoef(labels[::3], final_pred)
    sensitivity = tp/(tp+fn) if (tp+fn) > 0 else 0
    specificity = tn/(tn+fp) if (tn+fp) > 0 else 0
    ppv = tp/(tp+fp) if (tp+fp) > 0 else 0
    npv = tn/(tn+fn) if (tn+fn) > 0 else 0

    # Calculate AUC 
    auc = roc_auc_score(labels[::3], probabilities[::3]) if probabilities is not None else None

    # Create results dictionary
    result = {
        'accuracy': accuracy,
        'F1': f1,
        'AUC': auc,
        'MCC': mcc,
        'Kappa': kappa,
        'Sensitibity': sensitivity,
        'Specificity':specificity,
        'PPV': ppv,
        'NPV': npv              
    }
    if grid_search is not None:
        result['Best_Param'] = grid_search.best_params_
    
    else:
        result['Best_Param'] = "-"
    return result

In [6]:
def classification(data, features, features_names, classifier, classifier_name, param_grid=None):
    '''
    Function to train and evaluate different datasets using a specified classifier.
    Evaluation uses Leave-One-Group-Out cross-validation method.
    If a param_grid is supplied, it performs hyperparameter tuning to maximize the MCC score.
    Returns two dataframes:
        - train_results_df: Metrics for training sets.
        - test_results_df: Metrics for test sets (validation data).
    '''
    grup = data["id"]
    Class = data.iloc[:, 754]
    train_results_dict = {}
    test_results_dict = {}

    # Leave-One-Group-Out cross-validation
    logo = LeaveOneGroupOut()

    for num, feature_set in enumerate(features):
        sel_features = data[feature_set]

        # Predictions, labels and probabilities
        train_pred = []
        test_pred = []
        train_labels=[]
        test_labels = []
        
        train_probab = []
        test_probab = []
        
        best_model = None

        # Cross-validation LOGO
        for train_idx, test_idx in logo.split(sel_features, Class, groups=grup):
            X_train, X_test = sel_features.iloc[train_idx], sel_features.iloc[test_idx]
            y_train, y_test = Class.iloc[train_idx], Class.iloc[test_idx]

            if param_grid is not None:
                # Perform GridSearchCV
                mcc_score = make_scorer(matthews_corrcoef)
                grid_search = GridSearchCV(classifier, param_grid, scoring=mcc_score, n_jobs=-1)
                grid_search.fit(X_train, y_train)
                
                best_model = grid_search.best_estimator_

                # Predictions
                train_predictions = best_model.predict(X_train)
                test_predictions = best_model.predict(X_test)                
                
                if "predict_proba" in dir(best_model):            
                    train_probab.extend(best_model.predict_proba(X_train)[:, 1])
                    test_probab.extend(best_model.predict_proba(X_test)[:, 1])
                
                elif "decision_function" in dir(best_model):
                    train_probab.extend(best_model.decision_function(X_train))
                    test_probab.extend(best_model.decision_function(X_test))
                     
            else:
                # Train classifier without hyperparameter tuning
                classifier.fit(X_train, y_train)
                train_predictions = classifier.predict(X_train)
                test_predictions = classifier.predict(X_test)
                
                best_model = classifier
                
                if "predict_proba" in dir(classifier):            
                    train_probab.extend(classifier.predict_proba(X_train)[:, 1])
                    test_probab.extend(classifier.predict_proba(X_test)[:, 1])
                
                elif "decision_function" in dir(classifier):
                    train_probab.extend(classifier.decision_function(X_train))
                    test_probab.extend(classifier.decision_function(X_test))
                    
            # Save predictions
            train_pred.extend(train_predictions)
            test_pred.extend(test_predictions)
            train_labels.extend(y_train)
            test_labels.extend(y_test)
       
        # Save trained model
        model_path = f"Models/{classifier_name}_{features_names[num]}.pkl"
        joblib.dump(best_model, model_path)
        
        # Calculate metrics for train and test
        train_metrics = metrics_calculation(train_labels, train_pred, train_probab, grid_search if param_grid else None)
        test_metrics = metrics_calculation(test_labels, test_pred, test_probab, grid_search if param_grid else None)
        
        # Store results
        train_results_dict[features_names[num]] = train_metrics
        test_results_dict[features_names[num]] = test_metrics

    # Convert results to DataFrame
    train_results_df = pd.DataFrame.from_dict(train_results_dict, orient='index')
    train_results_df.index.name = 'Feature_Type'

    test_results_df = pd.DataFrame.from_dict(test_results_dict, orient='index')
    test_results_df.index.name = 'Feature_Type'

    return train_results_df, test_results_df

## K- Nearest Neighbors (k-NN)

The k-NN classifier model is used. Performance is evaluated using different number of neighbors. The number of neighbors is optimized during the cross-validation step for each subset of variables / components to be tested.

In [9]:
# Define classifier
knn = KNeighborsClassifier()

# Parameters grid
param_grid_knn = {'n_neighbors': [1,3,5,7,11,15]}

# Classification and performance metrics calculation
train_knn_results, knn_results = classification(all_data , all_features, all_names, knn, "KNN", param_grid_knn)

## Naive Bayes

The classification performance for each subset of variables/components is evaluated using the Naive Bayes classifier.

In [8]:
# Classifier
nb = GaussianNB()

# Classification and performance metrics calculation
train_nb_results, nb_results = classification(all_data , all_features, all_names, nb, "NaiveBayes")

## Logistic Regression

The classification performance for each subset of variables/components is evaluated using the Logistic Regression classifier.

In [10]:
# Classifier
lr = LogisticRegression(max_iter=200) #increase max number of iterations

# Classification and performance metrics calculation
train_lr_results, lr_results = classification(all_data , all_features, all_names, lr, "LogisticRegression")

## Support Vector Machine (SVM)

The SVM classifier model is used with both a linear kernel and an RBF kernel. Performance is evaluated using different values of `C`and `gamma`. These parameters are optimized during the cross-validation step for each subset of variables / components to be tested.

In [11]:
# Classifiers
svm_l = SVC(kernel = "linear", probability=True, class_weight="balanced")
svm_r = SVC(kernel = "rbf", probability=True, class_weight="balanced")

# Parameters grid
param_grid_svm = {
    'C': [0.01, 0.1, 0.5, 1,10,100],
    'gamma': ['scale', 'auto']
}

# Classification and performance metrics calculation
# Linear model
train_svm_l_results, svm_l_results = classification(all_data , all_features, all_names, svm_l, "SVM_Linear", param_grid_svm)

#RBF model
train_svm_r_results, svm_r_results = classification(all_data , all_features, all_names, svm_r, "SVM_RBF",param_grid_svm)

## Random Forest

A Random Forest classifier model is used. Performance is evaluated using different numbers of trees (estimators), reducing the maximum depth to 5 or 10, and increasing the minimum samples split to 5 or 10 to reduce overfitting. These parameters are optimized during the cross-validation step for each subset of variables/components to be tested, with the goal of maximizing the MCC score.

In [12]:
# Classifier
rf = RandomForestClassifier(class_weight="balanced",n_jobs=-1, random_state=123) 

# Parameters grid
param_grid_rf = {
    'n_estimators': [20,50,100,200],
    'max_depth': [5,10],
    'min_samples_split':[5,10] }

# Classification and performance metrics calculation
train_rf_results, rf_results = classification(all_data , all_features, all_names, rf, "RandomForest",param_grid_rf)

## Artificial Neural Network

For evaluating an artificial neural network, a similar strategy is employed by using each subset of variables/components obtained in the variable reduction step.    
In this ocasion, the `classification` function is not used since small code modifications are applied.   
First, a neural network is built using the `Sequential` function from `keras`. The architecture includes a dense hidden layer with ReLU activation and a dropout rate of 0.2 to prevent overfitting. The output layer consists of a single neuron with sigmoid activation designed for a binary classification. The model is compiled using the `binary crossentropy` loss function, and the AUC metric for optimization.       
For de cross-validation process, Stratified Group K Fold with 10 splits is used to ensure balanced class distributions within each fold. Hyperparameters optimization is applied over the provided parameter grid to maximize the MCC score on the validation data. Parameters to be optimized include the number of neurons in the hidden layer, and the type of optimizer. The maximum number of epochs is set to 100, with early stopping implemented to finish training if there is no improvement in the model's performance for 10 consecutive epochs.   
Performance is assessed using the `metrics_calculation` function, which calculates different evaluation materics as previously explained.
Finally, two dataframes are generated: one containing the metrics of the training process, and the other one with the metrics of the test set (validation data).

In [13]:
# Define neural network architecture:
def neural_network(neurons):
    tf.random.set_seed(123)
    nn = Sequential()   
    nn.add(Dense(neurons, activation="relu", input_shape=(sel_features.shape[1],)))  
    nn.add(Dropout(0.2))
    nn.add(Dense(1, activation="sigmoid"))  # Binary output
    nn.compile(loss="binary_crossentropy", metrics=["AUC"])
    return nn


# Early stopping
early_stopping = EarlyStopping(
    monitor="val_loss",  
    patience=10,         
    restore_best_weights=True 
)

# Compile model
nn_model = KerasClassifier(model=neural_network, verbose=0, epochs=100, random_state=123,
                          callbacks=[early_stopping])


# Parameter grid
param_grid = {"model__neurons": [8,16,32,64,128], 
             "optimizer": ["adam", "sdg", "rmsprop"]} 

In [ ]:
grup = all_data["id"]
Class = all_data.iloc[:, 754]
train_results_dict = {}
test_results_dict = {}

# Stratified Group K Fold
sgkf = StratifiedGroupKFold(n_splits=10)

# Evaluate each set of features
for num,feature_set in enumerate(all_features):
    sel_features = all_data[feature_set]
    results = []        

    # List of predictions and labels
    train_pred = []
    test_pred = []
    train_labels=[]
    test_labels = []

    train_probab = []
    test_probab = []    
        
    # Cross-validation SGKF
    for train_idx, test_idx in sgkf.split(sel_features, Class, groups=grup):
        X_train, X_test = sel_features.iloc[train_idx], sel_features.iloc[test_idx]
        y_train, y_test = Class.iloc[train_idx], Class.iloc[test_idx]
               
        # Search for best params
        mcc_score = make_scorer(matthews_corrcoef)
        grid_search = GridSearchCV(nn_model, param_grid, scoring= mcc_score,  n_jobs=-1)
        grid_search.fit(X_train, y_train, validation_data=(X_test, y_test))         
        
        best_model = grid_search.best_estimator_
        
        # Train and test using best params
        train_predictions = best_model.predict(X_train)
        test_predictions = best_model.predict(X_test)

        # Save probabilitites for AUC calcualtion
        train_probab.extend(best_model.predict_proba(X_train)[:, 1])
        test_probab.extend(best_model.predict_proba(X_test)[:, 1])
                
        # Save predictions
        train_pred.extend(train_predictions)
        train_labels.extend(y_train)
        test_pred.extend(test_predictions)
        test_labels.extend(y_test)
                
    #Save model
    model_path = f"Models/NeuralNetwork_{all_names[num]}.pkl"
    joblib.dump(best_model, model_path)
    
    # Calculate metrics for train and test
    train_metrics = metrics_calculation(train_labels, train_pred, train_probab, grid_search if param_grid else None)
    test_metrics = metrics_calculation(test_labels, test_pred, test_probab, grid_search if param_grid else None)

    # Store results
    train_results_dict[all_names[num]] = train_metrics
    test_results_dict[all_names[num]] = test_metrics

# Convert results to DataFrame
train_nn_results = pd.DataFrame.from_dict(train_results_dict, orient='index')
train_nn_results.index.name = 'Feature_Type'

nn_results = pd.DataFrame.from_dict(test_results_dict, orient='index')
nn_results.index.name = 'Feature_Type'

## Saving results

The performance metrics for both the training and test (validation) data from the optimized models of each classifier are saved.

In [15]:
#Save train results
train_knn_results.to_csv("Results/train_knn_results.csv", sep=";")
train_nb_results.to_csv("Results/train_nb_results.csv", sep=";")
train_lr_results.to_csv("Results/train_lr_results.csv", sep=";")
train_svm_l_results.to_csv("Results/train_svm_l_results.csv", sep=";")
train_svm_r_results.to_csv("Results/train_svm_r_results.csv", sep=";")
train_rf_results.to_csv("Results/train_rf_results.csv", sep=";")
train_nn_results.to_csv("Results/train_nn_results.csv", sep=";")

#Save results
knn_results.to_csv("Results/knn_results.csv", sep=";")
nb_results.to_csv("Results/nb_results.csv", sep=";")
lr_results.to_csv("Results/lr_results.csv", sep=";")
svm_l_results.to_csv("Results/svm_l_results.csv", sep=";")
svm_r_results.to_csv("Results/svm_r_results.csv", sep=";")
rf_results.to_csv("Results/rf_results.csv", sep=";")
nn_results.to_csv("Results/nn_results.csv", sep=";")